In [2]:
import datetime
import numpy as np
from influxdb import InfluxDBClient

In [3]:
client = InfluxDBClient(host='140.112.42.160', port=8086, database='test_2')

In [4]:
CODE_TO_COUNTRY = {              # country language code to full language name  
        'zh-tw': 'Taiwan',
        'ko': 'Korean', 
        'ja': 'Japan',
        'es': 'Spanish',
        'fr': 'French',
        'ru': 'Russian',
        'west-us': 'West US',
        'US': 'United States',
        'SG': 'Singapore',
        'GB': 'Great Britain',
        'NL': 'Netherlands',
        'HK': 'Hong Kong'
    }

In [5]:
def build_query(t_s, t_e, channel=None, lang=None, loc=None):
    """
    gets all data from month/date to month/(date+1)
    t_s: time start
    t_e: time end 
    lang: stream language
    loc: client location 
    """
    channel = '/.*/' if channel is None else channel
    q = f"SELECT viewer, client_location, ip_list, fq_count, num_edge FROM {channel} WHERE time >= '{t_s}' AND time < '{t_e}'"
    if lang:
        q = f"{q} AND stream_language = '{lang}'"
    if loc:
        q = f"{q} AND client_location = '{loc}'"
    return q

def get_edge_num(result):
    tmp = list()
    transactions = 0
    for (stream, _), points in result.items():
        for point in points:
            edges = point['ip_list'].split(',')
            transactions += sum([int(fq) for fq in point['fq_count'].split(',')])
            for edge in edges:
                if edge not in tmp:
                    tmp.append(edge)
    return set(tmp), transactions

def mean(l):
    return sum(l)/len(l)

def TIMEFMT(t):
    return datetime.datetime.strptime(t, '%Y-%m-%dT%H:%M:%SZ')

#### Calculate gorund truth

In [8]:
START_T = '2019-11-01T00:00:00Z' # start time
END_T = '2019-11-02T00:00:00Z'   # end time
q = build_query(START_T, END_T, loc='tw', lang='zh-tw')
result = client.query(q)

ground_truth, _ = get_edge_num(result) # total # of edge servers
print(len(ground_truth))

37


In [9]:
START_T = '2019-11-01T10:00:00Z' # start time
END_T = '2019-11-01T11:00:00Z'   # end time
q = build_query(START_T, END_T, loc='tw', lang='zh-tw')
result = client.query(q)

M, _ = get_edge_num(result) # number of edge servers in first capture
print(len(M))

22


In [11]:
START_T = '2019-11-01T20:00:00Z' # start time
END_T = '2019-11-01T21:00:00Z'   # end time
q = build_query(START_T, END_T, loc='tw', lang='zh-tw')
result = client.query(q)

C, _ = get_edge_num(result) # number of edge servers in second capture
print(len(C))

20


#### Calculate intersection of edge servers between two captures 

In [12]:
R = M.intersection(C)
print(len(R))

13


#### Estimate total number of servers by CRC: $N=\dfrac{MC}{R}$

In [13]:
N = (len(M)*len(C))//len(R)
print(N)

33


#### Input whole week data 

In [14]:
def get_estimation(date, date_1, t1, t2, loc, lang):
    st = f'2019-11-{date}T00:00:00Z' 
    et = f'2019-11-{date_1}T00:00:00Z'
    m_st = f'2019-11-{date}T08:00:00Z'
    m_et = f'2019-11-{date}T{t1}Z'
    c_st = f'2019-11-{date}T16:00:00Z'
    c_et = f'2019-11-{date}T{t2}Z'
    
    q = build_query(st, et, loc=loc, lang=lang)
    result = client.query(q)
    gt, gt_t = get_edge_num(result) 
    
    q = build_query(m_st, m_et, loc=loc, lang=lang)
    result = client.query(q)
    M, M_t = get_edge_num(result) 
    
    q = build_query(c_st, c_et, loc=loc, lang=lang)
    result = client.query(q)
    C, C_t = get_edge_num(result)
    
    R = M.intersection(C)
    
    try:
        N = len(M)*len(C)/len(R)
    except ZeroDivisionError:
        N = 0
    
    return int(N), len(gt), gt_t, (M_t+C_t)

In [15]:
date =   ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11']
date_1 = ['02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
time_1 = ['08:15:00', '08:30:00', '09:00:00', '10:00:00', '12:00:00', '14:00:00', '16:00:00']
time_2 = ['16:15:00', '16:30:00', '17:00:00', '18:00:00', '20:00:00', '22:00:00', '23:59:59']

In [ ]:
gt = [37, 37, 37, 38, 38, 70, 36, 36, 36, 38, 37]
est = [[], [], [], [], [], [], []]
trans = [[], [], [], [], [], [], []]
error = []
error_daily = [[], [], [], [], [], [], []]

for i, (t1, t2) in enumerate(zip(time_1, time_2)):
    err = 0
    for d, d1 in zip(date, date_1):
        N, gt, gt_t, t = get_estimation(d, d1, t1, t2, 'tw', 'zh-tw')
        print(N, gt, t, gt_t)
        
        est[i].append(N)
        trans[i].append(t)
        
        err += (N-gt)**2
        error_daily[i].append(N-gt)
    err = (err/11)**(1/2)
    print(err)
    error.append(err)

30 37 716 18157
36 37 849 18081
33 37 641 15637
32 38 635 17637
42 38 595 18398
43 70 655 16896
35 36 717 15602
27 36 641 10780
39 36 620 12891
31 38 598 12968
34 37 714 10940
9.515537533185109
30 37 1398 18157
36 37 1787 18081
35 37 1301 15637
34 38 1346 17637
39 38 1226 18398
82 70 1437 16896
33 36 1378 15602
29 36 1237 10780
38 36 1209 12891
30 38 1289 12968
35 37 1362 10940
5.60032466591325
31 37 2072 18157
36 37 2626 18081
35 37 1913 15637
34 38 2033 17637
38 38 1844 18398
82 70 2046 16896
34 36 2001 15602
30 36 1812 10780
38 36 1746 12891
30 38 1882 12968
36 37 1924 10940
5.308655025693248
38 37 4007 18157
36 37 4879 18081
35 37 3805 15637
36 38 3655 17637
36 38 3529 18398
79 70 3825 16896
34 36 3907 15602
33 36 3441 10780
38 36 3477 12891
31 38 3776 12968
36 37 3786 10940
3.8376128944009875
37 37 6935 18157
36 37 7922 18081
37 37 6504 15637
36 38 6776 17637
37 38 6242 18398
77 70 5430 16896
33 36 6014 15602
33 36 4851 10780
36 36 5352 12891
33 38 5943 12968
35 37 4795 10940
3.04

In [ ]:
# import plotting related libraries
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import colors
import matplotlib.patches as mpatches

In [ ]:
gt = [37, 37, 37, 38, 38, 70, 36, 36, 36, 38, 37]
fig, ax = plt.subplots(figsize=(20, 10))
# x_axis = list(range(len(gt)))
x_axis = ['11/1','11/2', '11/3', '11/4', '11/5', '11/6', '11/7', '11/8', '11/9', '11/10', '11/11']
labels = ['8', '6', '4', '2', '1', '0.5', '0.25']
ax.set_xlabel('Date', fontsize=28)
ax.set_ylabel('Number of edge servers', fontsize=28)

color_idx = np.linspace(0, 1, 7)
plt.plot(x_axis, gt, linewidth=5, color='black', label='Ground Truth', alpha=0.7)
for i, (c, e) in enumerate(zip(color_idx, est[::-1])):
    plt.plot(x_axis, e, color=plt.cm.summer(c), label=labels[i], alpha=0.7, linewidth=3)

#lns = ln1+ln2
#labs = [l.get_label() for l in lns]
#ax1.legend(lns, labs, loc='center right', fontsize = 12)

# plt.title('CMR estimation results', fontsize=30)
ax.grid()
#leg = ax.legend(prop={'size': 12})
#leg.set_title('Sampling Duration', prop={'size': 14, 'weight': 'heavy'})
# plt.legend(title='Sampling Duration', fontsize=20)
leg = ax.legend(fontsize=20)
leg.set_title('Sampling Duration',prop={'size':20})

ax.tick_params(axis='both', which='major', labelsize=28)

plt.show()


In [ ]:
fig, ax = plt.subplots(figsize=(40, 20))
x_axis = ['0.25', '0.5', '1', '2', '4', '6', '8']

ax.set_xlabel('capture duration', fontsize=28)
ax.set_ylabel('RMSE', fontsize=28)

color_idx = np.linspace(0, 1, 7)
plt.plot(x_axis, error, color=plt.cm.summer(1), alpha=0.7)
plt.title('RMSE v.s Capture Duration', fontsize=40)

plt.show()

#### TODO: Plot # of reduction in transactions

In [ ]:
trans_gt = [18157, 18081, 15637, 17637, 18398, 16896, 15602, 10780, 12891, 12968, 10940]
fig, ax = plt.subplots(figsize=(40, 20))
x_axis = ['11/1','11/2', '11/3', '11/4', '11/5', '11/6', '11/7', '11/8', '11/9', '11/10', '11/11']
labels = ['8', '6', '4', '2', '1', '0.5', '0.25']
ax.set_xlabel('date', fontsize=28)
ax.set_ylabel('# of transactions', fontsize=28)

color_idx = np.linspace(0, 1, 7)
plt.plot(x_axis, trans_gt, linewidth=5, color='black', label='ground truth', alpha=0.7)
for i, (c, e) in enumerate(zip(color_idx, trans[::-1])):
    plt.plot(x_axis, e, color=plt.cm.summer(c), label=labels[i], alpha=0.7)
    
plt.title('Comparison in number of transactions', fontsize=40)
plt.legend()
plt.show()

In [ ]:
trans_date = list(zip(*trans))
per = [[round(t/gt, 3) for t in td] for (gt, td) in zip(trans_gt, trans_date)]
per = list(zip(*per)) 
print(per)

#### Box plot of *error rate* and *reduction percentage*

In [ ]:
error_daily = [tuple([abs(e)for e in err]) for err in error_daily]
error_daily_per = [[e/gt[i] for i, e in enumerate(err)] for err in error_daily]

In [ ]:
print(per)
print(error_daily)
print(error_daily_per)

In [ ]:
per_mean = [100*mean(l) for l in per]
error_daily_mean = [mean(l) for l in error_daily]
error_daily_per_mean = [100*mean(l) for l in error_daily_per]

print(per_mean)
print(error_daily_mean)
print(error_daily_per_mean)

In [ ]:
fig, ax1 = plt.subplots(figsize=(20, 10))
labels = ['0.25', '0.5', '1', '2', '4', '6', '8']

# mirror axis 
ax2 = ax1.twinx()

ln1 = ax1.plot(labels, per_mean, label='Percentage of Transactions', linestyle=':', linewidth=4, color='orange', marker='o', markersize=15)
ln2 = ax2.plot(labels, error_daily_mean, label='Average Error', linestyle='--', linewidth=4, color='green', marker='v', markersize=15)

# make labels appear in one legend
# https://stackoverflow.com/questions/5484922/secondary-axis-with-twinx-how-to-add-to-legend
lns = ln1+ln2
labs = [l.get_label() for l in lns]
ax1.legend(lns, labs, loc='center right', fontsize = 'xx-large')

ax2.set_ylabel('Average Error(# of servers)', fontsize=20)
ax1.set_ylabel('Percentage of Transactions(%)', fontsize=20)
ax1.set_xlabel('Sampling Duration(hrs)', fontsize=20)
# ax1.set_title('Relation between PoT and AE v.s Sampling Duration', fontsize=24)

ax1.grid()
plt.show()

In [ ]:
fig, ax1 = plt.subplots(figsize=(20, 10))
labels = ['0.25', '0.5', '1', '2', '4', '6', '8']

# mirror axis 
ax2 = ax1.twinx()

ln1 = ax1.plot(labels[::-1], per_mean[::-1], label=r'$R_{o}$', linestyle=':', linewidth=4, color='orange', marker='o', markersize=15)
ln2 = ax2.plot(labels[::-1], error_daily_per_mean[::-1], label=r'$R_{e}$', linestyle='--', linewidth=4, color='green', marker='v', markersize=15)

# make labels appear in one legend
# https://stackoverflow.com/questions/5484922/secondary-axis-with-twinx-how-to-add-to-legend
lns = ln1+ln2
labs = [l.get_label() for l in lns]
ax1.legend(lns, labs, loc='center right', fontsize = 28)

ax2.set_ylabel('Error Rate(%)', fontsize=28)
ax1.set_ylabel('Transaction Reduction Rate(%)', fontsize=28)
ax1.set_xlabel('Sampling Duration(hrs)', fontsize=28)
# ax1.set_title(r'Relation between $R_{o}$ and $R_{e}$ v.s Sampling Duration', fontsize=30)

ax1.tick_params(axis='both', which='major', labelsize=28)
ax2.tick_params(axis='both', which='major', labelsize=28)
ax1.grid()
plt.show()

In [ ]:
fig, ax1 = plt.subplots(figsize=(20, 10))
ax2 = ax1.twinx()

ax1.boxplot(per, 0, '')
ax2.boxplot(error_daily, 0, '')


plt.show()